In [42]:
import numpy as np
import pandas
import geopandas
from shapely.geometry import Point
from IPython.display import display, clear_output
import pptk
from sklearn.neighbors import KDTree
import time

In [74]:
def pointRound(x, base=5):
    return base * round(x/base)

In [79]:
def cluster(point_array, base=5):
    tmp = 0
    counter = 0

    lidar_points_clustered = np.copy(lidar_points)

    length = lidar_points_clustered.shape[0]

    for i in range(0, length):

        if(tmp == 9000):
            tmp = 0
            clear_output(wait=True)
            display("progress: " + str(counter / length))

        counter += 1
        tmp += 1

        lidar_points_clustered[i][0] = pointRound(lidar_points_clustered[i][0], base)
        lidar_points_clustered[i][1] = pointRound(lidar_points_clustered[i][1], base)
        lidar_points_clustered[i][2] = pointRound(lidar_points_clustered[i][2], base)

    #lidar_points = np.unique(lidar_points)

    print("before, there were: " + str(length) + " points")

    unique = np.unique(lidar_points_clustered, axis=0)
    
    print("afterwards, there were: " + str(unique.shape[0]) + " points")
    
    return unique

In [4]:
def isInBox(point_x, point_y , bounds):
    #bounds = (minx, miny, maxx, maxy)
    return ((point_x > bounds[0]) and (point_x < bounds[2]) and (point_y > bounds[1]) and (point_y < bounds[3]))

In [5]:
def crossProdTest(point_a, point_b, point_c):
    if ((point_a[1] == point_b[1]) and (point_b[1] == point_c[1])):
        if ((point_b[0] <= point_a[0]) and (point_a[0] <= point_c[0]) or (point_c[0] <= point_a[0]) and (point_a[0] <= point_b[0])):
            return 0
        else:
            return 1
        
    if ((point_a[1] == point_b[1]) and (point_a[0] == point_b[0])):
        return 0
    
    if (point_b[1] > point_c[1]):
        point_b, point_c = point_c, point_b
    
    if ((point_a[1] <= point_b[1]) or (point_a[1] > point_c[1])):
        return 1
    
    delta = (point_b[0] - point_a[0]) * (point_c[1] - point_a[1]) - (point_b[1] - point_a[1]) * (point_c[0] - point_a[0])
    
    if (delta > 0):
        return -1
    elif (delta < 0):
        return 1
    else:
        return 0

In [6]:
def isPointInPolygon(point, polygon):
    #polygon = np.insert(polygon, 0, polygon[-1])    
    polygon = [polygon[-1]] + polygon
    t = -1
    
    #print("poly: " + str(polygon))
    
    for i in range(0, (len(polygon) - 1)):
        t = t * crossProdTest(point, polygon[i], polygon[i + 1])
        if (t == 0):
            return True
    return (t >= 0)
    

In [7]:
shape_file = "data/colonge_small/shp_koeln/gis_osm_buildings.shp"
shape  = geopandas.read_file(shape_file)

transport_shape = "./data/colonge_small/shp_koeln/gis_osm_transport.shp"
transport_shape = geopandas.read_file(transport_shape)
transport_shape.drop( transport_shape[ transport_shape['fclass'] == 'taxi' ].index , inplace=True)

lidar_file = "data/colonge_small/dom1l-fp_32356_5645_1_nw.xyz"
#read the file into a numpy array
lidar_points = np.loadtxt(open(lidar_file, "r"), delimiter=",", skiprows=0)

In [32]:
max_index = shape.index.max()

list_polygons = []

for i in range(0, (max_index + 1)):
    current_object = shape.loc[i]
    
    shape_name = current_object['name']
    shape_type = current_object['type']
    
    if (shape_name == None and (shape_type == None or shape_type == "appartment" or shape_type == "house")):
        #print(current_object['geometry'])
        geometry_object = current_object['geometry']
        
        #print(geometry_object.size)

        list_polygons.append([geometry_object.exterior.coords[:], geometry_object.bounds, 0, True])
        
        pure = geometry_object.exterior.coords[:]
        
        #print(len(geometry_object.exterior.coords[:]))
        
        
        
# print((list_polygons[0]))

# print(isPointInPolygon((356619.9304537382,5.64553368694670775e6), list_polygons[0][0]))

#lidar_z_sorted_desc = np.flip(lidar_points[lidar_points[:,2].argsort()], 0)

In [80]:
clustered = cluster(lidar_points, 5)
lidar_clustered_z_sorted_desc = np.flip(clustered[clustered[:,2].argsort()], 0)

'progress: 0.9999635714545918'

before, there were: 8235300 points
afterwards, there were: 107496 points


In [94]:
max_distance = 50
standart_height = 80

length = len(list_polygons)

counter = 0

not_found = 0

for polygon in list_polygons:
    
    clear_output(wait=True)
    display("progress: " + str(counter / length))
    counter += 1
    
    #bounds = (minx, miny, maxx, maxy)
    mid_x = (polygon[1][0] + polygon[1][2]) / 2 #find the mid point for the x coordinate
    mid_y = (polygon[1][1] + polygon[1][3]) / 2 #find the mid point for the y coordinate
    nearest_neighbors = (tree.query_radius(np.array([mid_x, mid_y, standart_height]).reshape(1, -1), max_distance))
    
    nearest_neighbors = np.sort(nearest_neighbors[0])
    
    for neighbor_index in nearest_neighbors:
        neighbor_point = lidar_clustered_z_sorted_desc[neighbor_index]
        
        if(polygon[3]):
            if (isInBox(neighbor_point[0], neighbor_point[1], polygon[1])):
                if(isPointInPolygon(neighbor_point, polygon[0])):
                    polygon[2] = point[2]
                    polygon[3] = False
                    print(".")
                    break
    
    if(polygon[3]):
        #print("exhausted all " + str(len(nearest_neighbors)) + " but found not fitting one")
        not_found += 1

print("not found: " + str(not_found) + " of " + str(length))

'progress: 0.9995226730310263'

not found: 1024 of 2095


In [58]:
print(len(nearest_neighbors[0]))

15103


In [26]:
for polygon in list_polygons:
    if(polygon[3]):
        for point in lidar_z_sorted_desc:
            if (isInBox(point[0], point[1], polygon[1])):
                if(isPointInPolygon(point, polygon[0])):
                    polygon[2] = point[2]
                    polygon[3] = False
                    print(".")
                    break

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


KeyboardInterrupt: 